In [1]:
%load_ext pycodestyle_magic
%load_ext mypy_ipython
%pycodestyle_on

In [2]:
import doctest

In [3]:
import inspect
import types
import time


class MultiMethod:
    def __init__(self, name):
        self._methods = {}
        self.__name__ = name

    def register(self, method):
        sig = inspect.signature(method)
        types = []
        for name, param in sig.parameters.items():
            if name == 'self':
                continue

            if param.annotation is inspect.Parameter.empty:
                raise TypeError(
                    f'argument {name} must be annotated with a type')
            if not isinstance(param.annotation, type):
                raise TypeError(f'argument {name} must be a type')
            if param.default is not inspect.Parameter.empty:
                self._methods[tuple(types)] = method
            types.append(param.annotation)

        self._methods[tuple(types)] = method

    def __call__(self, *args):
        types = tuple(type(arg) for arg in args[1:])
        method = self._methods.get(types, None)
        if method:
            return method(*args)
        else:
            raise TypeError('no matching method for types {}'.format(types))

    def __get__(self, instance, cls):
        if instance is None:
            return self

        return types.MethodType(self, instance)


class MultiDict(dict):
    def __setitem__(self, key, value):
        if key in self:
            current_value = self[key]
            if isinstance(current_value, MultiMethod):
                current_value.register(value)
            else:
                mvalue = MultiMethod(key)
                mvalue.register(current_value)
                mvalue.register(value)
                super().__setitem__(key, mvalue)
        else:
            super().__setitem__(key, value)


class MultipleMeta(type):
    @classmethod
    def __prepare__(cls, clsname, bases):
        return MultiDict()

    def __new__(cls, clsname, bases, clsdict):
        return super().__new__(cls, clsname, bases, clsdict)


class Spam(metaclass=MultipleMeta):
    _fields = ('a', 'b')

    def bar(self, x: int, y: int):
        print('bar 1:', x, y)

    def bar(self, s: str, n: int = 0):
        print('bar 2:', s, n)


class Date(metaclass=MultipleMeta):
    def __init__(self, year: int, month: int, day: int):
        self.year = year
        self.month = month
        self.day = day

    def __init__(self):
        t = time.localtime()
        self.__init__(t.tm_year, t.tm_mon, t.tm_mday)


"""
Method overloading

    >>> s = Spam()
    >>> s.bar(2, 3)
    bar 1: 2 3
    >>> s.bar('hello')
    bar 2: hello 0
    >>> s.bar('hello', 5)
    bar 2: hello 5
    >>> s.bar(2, 'hello')
    Traceback (most recent call last):
        ...
    TypeError: no matching method for types (<class 'int'>, <class 'str'>)

Overloaded __init__

    >>> d = Date(2012, 12, 21)
    >>> d.year, d.month, d.day
    (2012, 12, 21)
    >>> e = Date()
    >>> e.year, e.month, e.day
    (2021, 2, 1)

"""

doctest.testmod()

TestResults(failed=0, attempted=9)

In [4]:
import types


class dispatch:
    def __init__(self, func):
        self._methods = {}
        self.__name__ = func.__name__
        self._default = func

    def register(self, *types):
        def register(func):
            num_defaults = len(func.__defaults__) if func.__defaults__ else 0
            for n in range(num_defaults + 1):
                self._methods[types[:len(types) - n]] = func
            return self
        return register

    def __call__(self, *args):
        types = tuple(type(arg) for arg in args[1:])
        method = self._methods.get(types, None)
        if method:
            return method(*args)
        else:
            return self._default(*args)

    def __get__(self, instance, cls):
        if instance is None:
            return self

        return types.MethodType(self, instance)


class Spam:
    @dispatch
    def bar(self, *args):
        raise TypeError(
            f'no matching method for {Spam.bar.__name__}')

    @bar.register(int, int)
    def bar(self, x, y):
        print('bar 1:', x, y)

    @bar.register(str, int)
    def bar(self, s, n=0):
        print('bar 2:', s, n)


"""

>>> s = Spam()
>>> s.bar(2, 3)
bar 1: 2 3
>>> s.bar('hello')
bar 2: hello 0
>>> s.bar('hello', 5)
bar 2: hello 5
>>> s.bar(2, 'hello')
Traceback (most recent call last):
    ...
TypeError: no matching method for bar
"""

doctest.testmod()

TestResults(failed=0, attempted=5)